In [66]:
import time


In [74]:
def set_direction(value):
    value = max(-100, min(100, value))
    pwd = str(int(value * 2000 + 1500000))
    with open('/dev/bone/pwm/1/b/duty_cycle', 'w') as filetowrite:
        filetowrite.write(pwd)

def set_speed(value):
    value = max(0, min(100, value))
    pwd = str(int(value * 4000 + 1550000))
    with open('/dev/bone/pwm/1/b/duty_cycle', 'w') as filetowrite:
        filetowrite.write(pwd)

In [61]:
# init PWM
# P9_14 - Speed/ESC
with open('/dev/bone/pwm/1/a/period', 'w') as filetowrite:
    filetowrite.write('20000000')
with open('/dev/bone/pwm/1/a/duty_cycle', 'w') as filetowrite:
    filetowrite.write('1550000')
with open('/dev/bone/pwm/1/a/enable', 'w') as filetowrite:
    filetowrite.write('1')
# P9_16 - Steering
with open('/dev/bone/pwm/1/b/period', 'w') as filetowrite:
    filetowrite.write('20000000')
with open('/dev/bone/pwm/1/b/duty_cycle', 'w') as filetowrite:
    filetowrite.write('1500000')
with open('/dev/bone/pwm/1/b/enable', 'w') as filetowrite:
    filetowrite.write('1')
time.sleep(10)


In [31]:
# test PWD
set_direction(100)
time.sleep(3)
set_direction(-100)
time.sleep(3)
set_direction(0)
time.sleep(3)
set_speed(20)
time.sleep(3)
set_speed(0)
